In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

## Question 1

### Dataframe

The dataframe will consist of three columns:

* PostalCode
* Borough
* Neighborhood

In [1]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
requestUrl = requests.get(url).text
soup = BeautifulSoup(requestUrl,'lxml')

In [6]:
table = soup.find('table')
fields = table.find_all('td')

postalcode = []
borough = []
neighborhood = []

In [8]:
for i in range(0, len(fields), 3):
    postalcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighborhood.append(fields[i+2].text.strip())
        
df = pd.DataFrame(data=[postalcode, borough, neighborhood]).transpose()
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Ignore cells with a borough that is **Not assigned**

In [9]:
df.drop(df[df['Neighborhood'] == 'Not assigned'].index, inplace = True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df.shape

(103, 3)

## Question 2

### Geographical coordinates of each postal code

In [11]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')

In [13]:
geo.columns=['PostalCode','Latitude','Longitude']

In [14]:
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
newdf = pd.merge(df,
                 geo[['PostalCode','Latitude','Longitude']],
                 on='PostalCode')
newdf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [16]:
newdf.shape

(103, 5)

## Question 3
### Explore and cluster the neighborhoods in Toronto

In [21]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [22]:
!pip install folium

     |████████████████████████████████| 102kB 8.4MB/s ta 0:00:011


In [23]:
import folium

In [26]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto